In [1]:
import os
import glob

from IPython.display import display, Markdown
import pandas as pd

# Group types of models (experiment type and model type) and pick best performing in terms of f1-score per unit of analysis and report them in a table


In [2]:
results_filepaths = glob.glob('./logged_performance_per_model/*/*raw*.csv')

In [3]:
results_filepaths

['./logged_performance_per_model/distilbert-base-multilingual-cased/truncated_raw_truncated_single_instance_distilbert-base-multilingual-cased-title_and_first_sentence_each_paragraph_metrics.csv',
 './logged_performance_per_model/distilbert-base-multilingual-cased/truncated_raw_truncated_single_instance_distilbert-base-multilingual-cased-raw_text_metrics.csv',
 './logged_performance_per_model/distilbert-base-multilingual-cased/truncated_raw_truncated_single_instance_distilbert-base-multilingual-cased-title_and_5_sentences_metrics.csv',
 './logged_performance_per_model/distilbert-base-multilingual-cased/truncated_raw_truncated_single_instance_distilbert-base-multilingual-cased-title_metrics.csv',
 './logged_performance_per_model/distilbert-base-multilingual-cased/truncated_raw_truncated_single_instance_distilbert-base-multilingual-cased-title_and_first_paragraph_metrics.csv',
 './logged_performance_per_model/distilbert-base-multilingual-cased/truncated_raw_truncated_single_instance_dist

In [4]:
dfs_list = []
for results_filepath in results_filepaths:
    model_name = results_filepath.split('/')[-2]
    results_df_i = pd.read_csv(results_filepath)
    results_df_i['model_name'] = model_name
    dfs_list.append(results_df_i)

results_df = pd.concat(dfs_list).set_index(['language', 'model_name', 'unit_of_analysis']).sort_index()
results_df

f1_micro  \
language model_name              unit_of_analysis                                    
en       EleutherAI-gpt-neo-1.3B raw_text                                 0.693042   
                                 title                                    0.646067   
                                 title_and_10_sentences                   0.682451   
                                 title_and_5_sentences                    0.668524   
                                 title_and_first_paragraph                0.686981   
...                                                                            ...   
ru       xlm-roberta-large       title                                    0.463768   
                                 title_and_10_sentences                   0.533333   
                                 title_and_5_sentences                    0.463768   
                                 title_and_first_paragraph                0.488889   
                                 title_and_first_sentence_each_paragraph  0.520548   

                                                                          precision_micro  \
language model_name              unit_of_analysis                                           
en       EleutherAI-gpt-neo-1.3B raw_text                                        0.783951   
                                 title                                           0.759076   
                                 title_and_10_sentences                          0.792880   
                                 title_and_5_sentences                           0.776699   
                                 title_and_first_paragraph                       0.792332   
...                                                                                   ...   
ru       xlm-roberta-large       title                                           0.615385   
                                 title_and_10_sentences                          0.625000   
                                 title_and_5_sentences                           0.615385   
                                 title_and_first_paragraph                       0.673469   
                                 title_and_first_sentence_each_paragraph         0.633333   

                                                                          recall_micro  \
language model_name              unit_of_analysis                                        
en       EleutherAI-gpt-neo-1.3B raw_text                                     0.621027   
                                 title                                        0.562347   
                                 title_and_10_sentences                       0.599022   
                                 title_and_5_sentences                        0.586797   
                                 title_and_first_paragraph                    0.606357   
...                                                                                ...   
ru       xlm-roberta-large       title                                        0.372093   
                                 title_and_10_sentences                       0.465116   
                                 title_and_5_sentences                        0.372093   
                                 title_and_first_paragraph                    0.383721   
                                 title_and_first_sentence_each_paragraph      0.441860   

                                                                          f1_macro  \
language model_name              unit_of_analysis                                    
en       EleutherAI-gpt-neo-1.3B raw_text                                 0.507779   
                                 title                                    0.443437   
                                 title_and_10_sentences                   0.498518   
                                 title_and_5_sentences                    0.451412   
                                 title_and_first_paragraph

In [5]:
results_df.to_csv('performance_of_models.csv')

### Generate the tables to report

In [6]:
def display_performance_table(df, metric, index_cols=['model_name'], display_=True):
    report_table = df.reset_index().copy()
    report_table['result'] = report_table[f'{metric}'].map(lambda x: f'{x:.3f}')
    report_table['col_title'] = report_table.unit_of_analysis.str.split('_').str.join(' ') 
    report_table['col_title'] = pd.Categorical(
        report_table.col_title,
        categories=['title', 'title and first paragraph', 'title and 5 sentences', 'title and 10 sentences',
                    'title and first sentence each paragraph', 'raw text'],
        ordered=True)
    report_table = report_table[index_cols + ['col_title', 'result']]\
        .pivot_table(index=index_cols, columns=['col_title'], values=['result'], aggfunc='first', fill_value=0)\
        .droplevel(0, axis=1)

    report_table.columns.names = [None]

    # Highlight best scoring models according to their average
    mean_perf_arr = report_table.applymap(lambda x: float(str(x).split(' ')[0])).to_numpy()
    highlight_mask = mean_perf_arr == mean_perf_arr.max()
    report_table_arr = report_table.to_numpy()  # Note it passes the array by reference
    report_table_arr[highlight_mask] = '**' + report_table_arr[highlight_mask] + '**'

    if display_:
        display(Markdown(report_table.to_markdown()))
    
    return report_table

In [7]:
display_performance_table(df=results_df.loc['en'], metric='f1_micro', index_cols=['model_name'], display_=True)

| model_name                         |   title | title and first paragraph   |   title and 5 sentences |   title and 10 sentences |   title and first sentence each paragraph |   raw text |
|:-----------------------------------|--------:|:----------------------------|------------------------:|-------------------------:|------------------------------------------:|-----------:|
| EleutherAI-gpt-neo-1.3B            |   0.646 | 0.687                       |                   0.669 |                    0.682 |                                     0.709 |      0.693 |
| EleutherAI-gpt-neo-125M            |   0.573 | 0.647                       |                   0.642 |                    0.636 |                                     0.649 |      0.631 |
| bert-base-multilingual-cased       |   0.619 | 0.690                       |                   0.676 |                    0.689 |                                     0.688 |      0.711 |
| distilbert-base-multilingual-cased |   0.592 | 0.662                       |                   0.685 |                    0.686 |                                     0.684 |      0.684 |
| facebook-mbart-large-50            |   0.666 | **0.734**                   |                   0.731 |                    0.718 |                                     0.708 |      0.711 |
| gpt2                               |   0.625 | 0.664                       |                   0.678 |                    0.66  |                                     0.68  |      0.654 |
| xlm-roberta-large                  |   0.659 | 0.710                       |                   0.721 |                    0.71  |                                     0.709 |      0.7   |

,title,title and first paragraph,title and 5 sentences,title and 10 sentences,title and first sentence each paragraph,raw text
model_name,,,,,,
EleutherAI-gpt-neo-1.3B,0.646,0.687,0.669,0.682,0.709,0.693
EleutherAI-gpt-neo-125M,0.573,0.647,0.642,0.636,0.649,0.631
bert-base-multilingual-cased,0.619,0.690,0.676,0.689,0.688,0.711
distilbert-base-multilingual-cased,0.592,0.662,0.685,0.686,0.684,0.684
facebook-mbart-large-50,0.666,**0.734**,0.731,0.718,0.708,0.711
gpt2,0.625,0.664,0.678,0.660,0.680,0.654
xlm-roberta-large,0.659,0.710,0.721,0.710,0.709,0.700


### Generate tables for all languages

In [8]:
metrics_to_report = ['f1_micro', 'recall_micro', 'precision_micro', 'accuracy']

In [9]:
language_dict = {'en': 'English', 'it': 'Italian', 'fr': 'French', 'po': 'Polish', 'ru': 'Russian', 'ge': 'German'}

In [10]:
results_df

f1_micro  \
language model_name              unit_of_analysis                                    
en       EleutherAI-gpt-neo-1.3B raw_text                                 0.693042   
                                 title                                    0.646067   
                                 title_and_10_sentences                   0.682451   
                                 title_and_5_sentences                    0.668524   
                                 title_and_first_paragraph                0.686981   
...                                                                            ...   
ru       xlm-roberta-large       title                                    0.463768   
                                 title_and_10_sentences                   0.533333   
                                 title_and_5_sentences                    0.463768   
                                 title_and_first_paragraph                0.488889   
                                 title_and_first_sentence_each_paragraph  0.520548   

                                                                          precision_micro  \
language model_name              unit_of_analysis                                           
en       EleutherAI-gpt-neo-1.3B raw_text                                        0.783951   
                                 title                                           0.759076   
                                 title_and_10_sentences                          0.792880   
                                 title_and_5_sentences                           0.776699   
                                 title_and_first_paragraph                       0.792332   
...                                                                                   ...   
ru       xlm-roberta-large       title                                           0.615385   
                                 title_and_10_sentences                          0.625000   
                                 title_and_5_sentences                           0.615385   
                                 title_and_first_paragraph                       0.673469   
                                 title_and_first_sentence_each_paragraph         0.633333   

                                                                          recall_micro  \
language model_name              unit_of_analysis                                        
en       EleutherAI-gpt-neo-1.3B raw_text                                     0.621027   
                                 title                                        0.562347   
                                 title_and_10_sentences                       0.599022   
                                 title_and_5_sentences                        0.586797   
                                 title_and_first_paragraph                    0.606357   
...                                                                                ...   
ru       xlm-roberta-large       title                                        0.372093   
                                 title_and_10_sentences                       0.465116   
                                 title_and_5_sentences                        0.372093   
                                 title_and_first_paragraph                    0.383721   
                                 title_and_first_sentence_each_paragraph      0.441860   

                                                                          f1_macro  \
language model_name              unit_of_analysis                                    
en       EleutherAI-gpt-neo-1.3B raw_text                                 0.507779   
                                 title                                    0.443437   
                                 title_and_10_sentences                   0.498518   
                                 title_and_5_sentences                    0.451412   
                                 title_and_first_paragraph

In [11]:
def display_metrics_and_write_to_file(df, grouping_criterion, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    report_tables_dfs_dict = {metric: [] for metric in metrics_to_report}

    for language, results_df in df.groupby(level=0):
        display(Markdown(f'# {language_dict[language]}'))
        
        for metric in metrics_to_report:
            os.makedirs(os.path.join(output_dir, metric), exist_ok=True)

            output_dir_markdown = os.path.join(output_dir, metric, 'markdown')
            output_dir_latex = os.path.join(output_dir, metric, 'latex')
            output_dir_csv = os.path.join(output_dir, metric, 'csv')

            os.makedirs(output_dir_markdown, exist_ok=True)
            os.makedirs(output_dir_latex, exist_ok=True)
            os.makedirs(output_dir_csv, exist_ok=True)

            display(Markdown(f'## {metric}'))

            report_table = display_performance_table(df=results_df, index_cols=grouping_criterion, metric=metric, display_=True)

            # Export as markdown
            markdown_file = open(os.path.join(output_dir_markdown, f"{language_dict[language]}_{metric}.md"), "w")
            report_table.reset_index().to_markdown(markdown_file, index=False)
            markdown_file.close()

            # Export as latex table
            latex_file = open(os.path.join(output_dir_latex, f"{language_dict[language]}_{metric}.tex"), "w")
            report_table.reset_index().to_latex(latex_file, index=False)
            latex_file.close()

            # Export as csv
            report_table.to_csv(os.path.join(output_dir_csv, f"{language_dict[language]}_{metric}.csv"))

            # Stack all languages into single table
            report_table['language'] = language
            report_table = report_table.reset_index().set_index(['language'] + grouping_criterion)

            report_tables_dfs_dict[metric].append(report_table)

    # Report or store unified table
    display(Markdown(f'# All 6 Languages'))
    for metric in metrics_to_report:
        display(Markdown(f'## {metric}'))
        multi_language_report_table_metric = pd.concat(report_tables_dfs_dict[metric])
        display(Markdown(multi_language_report_table_metric.reset_index().to_markdown(index=False)))

        output_dir_markdown = os.path.join(output_dir, metric, 'markdown')
        output_dir_latex = os.path.join(output_dir, metric, 'latex')
        output_dir_csv = os.path.join(output_dir, metric, 'csv')

        # Export as markdown
        markdown_file = open(os.path.join(output_dir_markdown, f"all_6_languages_{metric}.md"), "w")
        multi_language_report_table_metric.reset_index().to_markdown(markdown_file, index=False)
        markdown_file.close()

        # Export as latex table
        latex_file = open(os.path.join(output_dir_latex, f"all_6_languages_{metric}.tex"), "w")
        multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)
        latex_file.close()

        # Export as csv
        multi_language_report_table_metric.to_csv(os.path.join(output_dir_csv, f"all_6_languages_{metric}.csv"))

# Per model type

In [12]:
display_metrics_and_write_to_file(df=results_df, grouping_criterion=['model_name'], output_dir='per_model_name_tables')

# English

## f1_micro

| model_name                         |   title | title and first paragraph   |   title and 5 sentences |   title and 10 sentences |   title and first sentence each paragraph |   raw text |
|:-----------------------------------|--------:|:----------------------------|------------------------:|-------------------------:|------------------------------------------:|-----------:|
| EleutherAI-gpt-neo-1.3B            |   0.646 | 0.687                       |                   0.669 |                    0.682 |                                     0.709 |      0.693 |
| EleutherAI-gpt-neo-125M            |   0.573 | 0.647                       |                   0.642 |                    0.636 |                                     0.649 |      0.631 |
| bert-base-multilingual-cased       |   0.619 | 0.690                       |                   0.676 |                    0.689 |                                     0.688 |      0.711 |
| distilbert-base-multilingual-cased |   0.592 | 0.662                       |                   0.685 |                    0.686 |                                     0.684 |      0.684 |
| facebook-mbart-large-50            |   0.666 | **0.734**                   |                   0.731 |                    0.718 |                                     0.708 |      0.711 |
| gpt2                               |   0.625 | 0.664                       |                   0.678 |                    0.66  |                                     0.68  |      0.654 |
| xlm-roberta-large                  |   0.659 | 0.710                       |                   0.721 |                    0.71  |                                     0.709 |      0.7   |

/tmp/ipykernel_17486/2331756451.py:30: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  report_table.reset_index().to_latex(latex_file, index=False)


## recall_micro

...

| language   | model_name                         | title     | title and first paragraph   | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   | raw text   |
|:-----------|:-----------------------------------|:----------|:----------------------------|:------------------------|:-------------------------|:------------------------------------------|:-----------|
| en         | EleutherAI-gpt-neo-1.3B            | 0.759     | 0.792                       | 0.777                   | 0.793                    | 0.786                                     | 0.784      |
| en         | EleutherAI-gpt-neo-125M            | 0.686     | 0.745                       | 0.739                   | 0.748                    | 0.742                                     | 0.706      |
| en         | bert-base-multilingual-cased       | 0.715     | 0.769                       | 0.775                   | 0.748                    | 0.718                                     | 0.785      |
| en         | distilbert-base-multilingual-cased | 0.700     | 0.752                       | 0.779                   | 0.765                    | 0.757                                     | 0.772      |
| en         | facebook-mbart-large-50            | 0.769     | 0.797                       | 0.812                   | 0.812                    | 0.778                                     | 0.777      |
| en         | gpt2                               | 0.700     | 0.713                       | 0.702                   | 0.732                    | 0.719                                     | 0.731      |
| en         | xlm-roberta-large                  | 0.765     | 0.805                       | **0.817**               | 0.796                    | 0.776                                     | 0.795      |
| fr         | EleutherAI-gpt-neo-1.3B            | 0.547     | 0.580                       | 0.538                   | 0.536                    | 0.562                                     | 0.571      |
| fr         | EleutherAI-gpt-neo-125M            | 0.402     | 0.462                       | 0.438                   | 0.446                    | 0.570                                     | 0.453      |
| fr         | bert-base-multilingual-cased       | 0.536     | 0.511                       | 0.559                   | 0.527                    | 0.600                                     | 0.598      |
| fr         | distilbert-base-multilingual-cased | 0.465     | 0.471                       | 0.505                   | 0.583                    | **0.619**                                 | 0.548      |
| fr         | facebook-mbart-large-50            | 0.536     | 0.579                       | 0.568                   | 0.553                    | 0.580                                     | 0.596      |
| fr         | gpt2                               | 0.419     | 0.477                       | 0.512                   | 0.475                    | 0.547                                     | 0.566      |
| fr         | xlm-roberta-large                  | 0.559     | 0.557                       | 0.544                   | 0.561                    | 0.575                                     | 0.553      |
| ge         | EleutherAI-gpt-neo-1.3B            | 0.705     | 0.700                       | 0.701                   | **0.796**                | 0.719                                     | 0.717      |
| ge         | EleutherAI-gpt-neo-125M            | 0.593     | 0.579                       | 0.649                   | 0.616                    | 0.657                                     | 0.622      |
| ge         | bert-base-multilingual-cased       | 0.597     | 0.681                       | 0.737                   | 0.709                    | 0.641                                     | 0.730      |
| ge         | distilbert-base-multilingual-cased | 0.605     | 0.664                       | 0.649                   | 0.654                    | 0.727                                     | 0.688      |
| ge         | facebook-mbart-large-50            | 0.709     | 0.676                       | 0.669                   | 0.739                    | 0.684                                     | 0.791      |
| ge         | gpt2                               | 0.543     | 0.591                       | 0.525                   | 0.699                    | 0.699                                     | 0.675      |
| ge         | xlm-roberta-large                  | 0.711     | 0.672                       | 0.717                   | 0.733                    | 0.732                                     | 0.733      |
| it         | EleutherAI-gpt-neo-1.3B            | 0.650     | 0.709                       | 0.717                   | 0.737                    | 0.731                                     | 0.770      |
| it         | EleutherAI-gpt-neo-125M            | 0.526     | 0.592                       | 0.574                   | 0.636                    | 0.681                                     | 0.623      |
| it         | bert-base-multilingual-cased       | 0.583     | 0.690                       | 0.686                   | 0.718                    | 0.667                                     | 0.742      |
| it         | distilbert-base-multilingual-cased | 0.571     | 0.617                       | 0.635                   | 0.713                    | 0.608                                     | 0.757      |
| it         | facebook-mbart-large-50            | 0.696     | 0.717                       | 0.730                   | 0.720                    | 0.748                                     | 0.778      |
| it         | gpt2                               | 0.561     | 0.588                       | 0.572                   | 0.637                    | 0.640                                     | 0.627      |
| it         | xlm-roberta-large                  | 0.711     | **0.785**                   | 0.755                   | 0.770                    | 0.699                                     | 0.778      |
| po         | EleutherAI-gpt-neo-1.3B            | 0.595     | 0.636                       | 0.705                   | 0.739                    | 0.766                                     | 0.746      |
| po         | EleutherAI-gpt-neo-125M            | 0.558     | 0.590                       | 0.592                   | 0.630                    | 0.681                                     | 0.609      |
| po         | bert-base-multilingual-cased       | 0.624     | 0.659                       | 0.715                   | 0.703                    | 0.750                                     | 0.745      |
| po         | distilbert-base-multilingual-cased | 0.576     | 0.655                       | 0.721                   | 0.727                    | 0.684                                     | 0.730      |
| po         | facebook-mbart-large-50            | 0.652     | 0.692                       | 0.733                   | 0.763                    | 0.796                                     | **0.801**  |
| po         | gpt2                               | 0.610     | 0.593                       | 0.602                   | 0.632                    | 0.678                                     | 0.688      |
| po         | xlm-roberta-large                  | 0.647     | 0.692                       | 0.718                   | 0.747                    | 0.775                                     | 0.779      |
| ru         | EleutherAI-gpt-neo-1.3B            | **0.727** | 0.581                       | 0.625                   | 0.600                    | 0.605                                     | 0.514      |
| ru         | EleutherAI-gpt-neo-125M            | 0.333     | 0.250                       | 0.375                   | 0.347                    | 0.308                                     | 0.190      |
| ru         | bert-base-multilingual-cased       | 0.500     | 0.478                       | 0.561                   | 0.480                    | 0.545                                     | 0.538      |
| ru         | distilbert-base-multilingual-cased | 0.410     | 0.414                       | 0.567                   | 0.567                    | 0.515                                     | 0.600      |
| ru         | facebook-mbart-large-50            | 0.609     | 0.600                       | 0.585                   | 0.633                    | 0.672                                     | 0.685      |
| ru         | gpt2                               | 0.231     | 0.174                       | 0.300                   | 0.118                    | 0.379                                     | 0.474      |
| ru         | xlm-roberta-large                  | 0.615     | 0.673                       | 0.615                   | 0.625                    | 0.633                                     | 0.636      |

/tmp/ipykernel_17486/2331756451.py:60: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)


## accuracy

| language   | model_name                         | title     | title and first paragraph   | title and 5 sentences   |   title and 10 sentences | title and first sentence each paragraph   | raw text   |
|:-----------|:-----------------------------------|:----------|:----------------------------|:------------------------|-------------------------:|:------------------------------------------|:-----------|
| en         | EleutherAI-gpt-neo-1.3B            | 0.058     | 0.136                       | 0.117                   |                    0.078 | 0.146                                     | 0.097      |
| en         | EleutherAI-gpt-neo-125M            | 0.068     | 0.087                       | 0.107                   |                    0.097 | 0.078                                     | 0.019      |
| en         | bert-base-multilingual-cased       | 0.097     | 0.136                       | 0.126                   |                    0.097 | 0.126                                     | 0.117      |
| en         | distilbert-base-multilingual-cased | 0.087     | 0.097                       | 0.117                   |                    0.058 | 0.068                                     | 0.087      |
| en         | facebook-mbart-large-50            | 0.097     | 0.126                       | **0.155**               |                    0.126 | 0.136                                     | 0.117      |
| en         | gpt2                               | 0.049     | 0.078                       | 0.087                   |                    0.068 | 0.058                                     | 0.039      |
| en         | xlm-roberta-large                  | 0.049     | 0.117                       | **0.155**               |                    0.126 | 0.068                                     | 0.097      |
| fr         | EleutherAI-gpt-neo-1.3B            | 0.024     | 0.095                       | 0.071                   |                    0.048 | 0.071                                     | 0.071      |
| fr         | EleutherAI-gpt-neo-125M            | 0.071     | 0.024                       | 0.048                   |                    0.071 | 0.048                                     | 0.000      |
| fr         | bert-base-multilingual-cased       | 0.048     | 0.095                       | 0.071                   |                    0.071 | **0.167**                                 | 0.048      |
| fr         | distilbert-base-multilingual-cased | 0.048     | 0.048                       | 0.048                   |                    0.048 | 0.048                                     | 0.071      |
| fr         | facebook-mbart-large-50            | 0.000     | 0.095                       | 0.024                   |                    0.024 | 0.095                                     | 0.071      |
| fr         | gpt2                               | 0.000     | 0.071                       | 0.024                   |                    0.024 | 0.071                                     | 0.071      |
| fr         | xlm-roberta-large                  | 0.095     | 0.071                       | 0.071                   |                    0.071 | 0.095                                     | 0.095      |
| ge         | EleutherAI-gpt-neo-1.3B            | 0.000     | 0.029                       | 0.000                   |                    0.057 | 0.029                                     | 0.057      |
| ge         | EleutherAI-gpt-neo-125M            | 0.000     | 0.000                       | 0.000                   |                    0     | 0.000                                     | 0.000      |
| ge         | bert-base-multilingual-cased       | 0.029     | 0.029                       | 0.086                   |                    0     | 0.000                                     | 0.000      |
| ge         | distilbert-base-multilingual-cased | 0.000     | 0.029                       | 0.000                   |                    0     | 0.029                                     | 0.029      |
| ge         | facebook-mbart-large-50            | 0.057     | **0.114**                   | 0.029                   |                    0.029 | 0.029                                     | 0.086      |
| ge         | gpt2                               | 0.029     | 0.000                       | 0.000                   |                    0     | 0.029                                     | 0.000      |
| ge         | xlm-roberta-large                  | 0.057     | 0.086                       | 0.029                   |                    0.029 | 0.057                                     | 0.029      |
| it         | EleutherAI-gpt-neo-1.3B            | 0.050     | 0.133                       | 0.117                   |                    0.183 | 0.133                                     | 0.200      |
| it         | EleutherAI-gpt-neo-125M            | 0.017     | 0.067                       | 0.033                   |                    0.05  | 0.067                                     | 0.067      |
| it         | bert-base-multilingual-cased       | 0.083     | 0.117                       | 0.100                   |                    0.167 | 0.100                                     | **0.267**  |
| it         | distilbert-base-multilingual-cased | 0.017     | 0.100                       | 0.117                   |                    0.1   | 0.033                                     | 0.117      |
| it         | facebook-mbart-large-50            | 0.117     | 0.117                       | 0.133                   |                    0.083 | 0.133                                     | 0.183      |
| it         | gpt2                               | 0.050     | 0.083                       | 0.050                   |                    0.083 | 0.100                                     | 0.050      |
| it         | xlm-roberta-large                  | 0.117     | 0.200                       | 0.167                   |                    0.15  | 0.117                                     | 0.200      |
| po         | EleutherAI-gpt-neo-1.3B            | 0.026     | 0.000                       | 0.026                   |                    0.026 | 0.051                                     | 0.051      |
| po         | EleutherAI-gpt-neo-125M            | 0.000     | 0.000                       | 0.000                   |                    0.051 | 0.000                                     | 0.026      |
| po         | bert-base-multilingual-cased       | 0.000     | 0.026                       | 0.051                   |                    0.026 | 0.051                                     | **0.103**  |
| po         | distilbert-base-multilingual-cased | 0.000     | 0.000                       | 0.051                   |                    0.026 | 0.051                                     | 0.026      |
| po         | facebook-mbart-large-50            | 0.000     | 0.026                       | 0.051                   |                    0.077 | 0.051                                     | 0.051      |
| po         | gpt2                               | 0.000     | 0.026                       | 0.000                   |                    0.026 | 0.051                                     | 0.026      |
| po         | xlm-roberta-large                  | 0.000     | 0.051                       | 0.026                   |                    0.026 | 0.077                                     | 0.077      |
| ru         | EleutherAI-gpt-neo-1.3B            | 0.053     | 0.079                       | 0.079                   |                    0.158 | 0.158                                     | 0.053      |
| ru         | EleutherAI-gpt-neo-125M            | 0.026     | 0.026                       | 0.053                   |                    0.053 | 0.105                                     | 0.026      |
| ru         | bert-base-multilingual-cased       | 0.105     | 0.132                       | 0.211                   |                    0.158 | **0.237**                                 | 0.211      |
| ru         | distilbert-base-multilingual-cased | 0.105     | 0.158                       | 0.132                   |                    0.211 | 0.132                                     | 0.184      |
| ru         | facebook-mbart-large-50            | 0.158     | 0.211                       | 0.132                   |                    0.211 | 0.158                                     | 0.184      |
| ru         | gpt2                               | 0.000     | 0.000                       | 0.026                   |                    0     | 0.079                                     | 0.026      |
| ru         | xlm-roberta-large                  | **0.237** | 0.211                       | 0.211                   |                    0.211 | 0.211                                     | 0.211      |

/tmp/ipykernel_17486/2331756451.py:60: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)
